
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>



# Exploding Join

In this lab, we will be working on improving query performance on an exploding join between 3 tables:
- **transactions**
- **stores**
- **countries**

We want to introduce and identify spill introduced by the exploding join and gradually improve the performance of the join.

#### Lab Notes
To successfully create a exploding join that spills to disk we had to specifically turn off the following optimizations:
- [Predictive optimization](https://docs.databricks.com/aws/en/optimizations/predictive-optimization) for Unity Catalog managed tables in the **lab** schema.
- Broadcast joins to gradually demonstrate the performance improvements of tuning the join strategy.

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default. If you use Serverless, errors will be returned when setting compute runtime properties.

Follow these steps to select the classic compute cluster:

1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

  - In the drop-down, select **More**.

  - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

1. Find the triangle icon to the right of your compute cluster name and click it.

1. Wait a few minutes for the cluster to start.

1. Once the cluster is running, complete the steps above to select your cluster.

## A. Classroom Setup

Run the following cell to configure your working environment for this course. It will also set your default catalog to your unique **labuser** catalog, and the default schema to **lab**. All tables will be read from and written to this location.
<br></br>

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course.

In [0]:
%run ./Includes/Classroom-Setup-4L

## B. Disable Caching

Run the following cell to set a Spark configuration variable that disables disk caching.

Turning disk caching off prevents Databricks from storing cloud storage files after the first query. This makes the effect of the optimizations more apparent by ensuring that files are always pulled from cloud storage for each query.

For more information, see [Optimize performance with caching on Databricks](https://docs.databricks.com/en/optimizations/disk-cache.html#optimize-performance-with-caching-on-databricks).

**NOTE:** This will not work in Serverless. Please use classic compute to turn off caching. If you're using Serverless, an error will be returned.

In [0]:
spark.conf.set('spark.databricks.io.cache.enabled', False)

## C. Creating & Storing Data

### C1. Create the transactions table
Generate the **transactions** table with the schema below and write it to a Delta table. This is the table with the largest amount of data, containing 2,000,000 rows.


In [0]:
from pyspark.sql.functions import *

## Delete the table if it exists
spark.sql('DROP TABLE IF EXISTS transactions')


## Create the table
transactions_df = (spark
                   .range(0, 2000000, 1, 32)
                    .select(
                        'id',
                        round(rand() * 10000, 2).alias('amount'),
                        (col('id') % 10).alias('country_id'),
                        (col('id') % 100).alias('store_id')
                    )
                    .write
                    .mode('overwrite')
                    .option("overwriteSchema", "true")
                    .saveAsTable('transactions')
                )

## Display the table
display(spark.sql('SELECT * FROM transactions LIMIT 10'))

### C2. Create the countries table
Generate the **countries** table to join with the **transactions** table later.

In [0]:
## Drop the table if it exists
spark.sql('DROP TABLE IF EXISTS countries')

## Create the table
countries = [(0, "Italy"),
             (1, "Canada"),
             (2, "Mexico"),
             (3, "China"),
             (4, "Germany"),
             (5, "UK"),
             (6, "Japan"),
             (7, "Korea"),
             (8, "Australia"),
             (9, "France"),
             (10, "Spain"),
             (11, "USA")
            ]
columns = ["id", "name"]

countries_df = (spark
                .createDataFrame(data = countries, schema = columns)
                .write
                .mode('overwrite')
                .saveAsTable("countries")
            )


## Display the table
display(spark.sql('SELECT * FROM countries'))


### C3. Create the stores table
- Generate the **stores** table to join with the **transactions** table later on.
- The **stores** table intentionally contains duplicates of the **id** value, which will introduce an exploding join with the **transactions** table.
- When joining the **transactions** table with the **stores** table, a number of rows will explode due to the duplicated **ids**.


In [0]:
## Drop the table if it exists
spark.sql('DROP TABLE IF EXISTS stores')


stores_df = (spark
                .range(0, 9999)
                .select(
                    (col('id') % 100).alias('id'), # intentionally duplicating ids to explode the join
                    round(rand() * 100, 0).alias('employees'),
                    (col('id') % 10).alias('country_id'),
                    expr('uuid()').alias('name')
                )
                .write
                .mode('overwrite')
                .saveAsTable('stores')
            )


## Display the table
display(spark.sql('SELECT * FROM stores ORDER BY id'))

## D. Perform an Exploding Join

In this cell, turn off broadcast joins to gradually demonstrate the performance improvements of tuning the join strategy.


- [spark.sql.autoBroadcastJoinThreshold](https://spark.apache.org/docs/3.5.3/sql-performance-tuning.html#other-configuration-options) documentation

- [spark.databricks.adaptive.autoBroadcastJoinThreshold](https://spark.apache.org/docs/3.5.3/sql-performance-tuning.html#converting-sort-merge-join-to-broadcast-join) documentation

In [0]:
## Disabling the automatic broadcast join entirely. That is, Spark will never broadcast any dataset for joins, regardless of its size.
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

# Disabling the broadcast join feature under AQE, meaning that even when using adaptive query execution, Spark will not attempt to broadcast any smaller side of a join.
spark.conf.set("spark.databricks.adaptive.autoBroadcastJoinThreshold", -1)

We will be joining the **transactions** table with the **countries** and **stores** data, and trigger the action by saving the result to a table named **transact_countries**. The query has been written for you below.

Run the cell, note down the time taken to execute the query, and compare it with each optimization.

**NOTE:** This should take about ~1 minute.


In [0]:
joined_df_nobroadcast = spark.sql("""
    SELECT 
        transactions.id,
        amount,
        countries.name as country_name,
        employees,
        stores.name as store_name
    FROM
        transactions
    JOIN
        stores
        ON
            transactions.store_id = stores.id
    JOIN
        countries
        ON
            transactions.country_id = countries.id
""")

(joined_df_nobroadcast
 .write
 .mode('overwrite')
 .saveAsTable('transact_countries')
)

### D1. TODO: See the Exploding Join
Open the Spark UI and navigate to the **Stages** page. Identify the explosion of rows in the DAG of the Spark UI. To view the DAG, complete the following:

1. In the cell above, expand **Spark Jobs**.

2. In the first job , right-click on **View** and select *Open in a New Tab*. 
    - **If the first job does not display the correct information, try the second one**.

    **NOTES:** In the Vocareum lab environment, if you click **View** without opening it in a new tab, the pop-up window will display an error.

3. In the new window, find the **Associated SQL Query** header at the top and select the number.

4. Here, you should see the entire query plan. Read the DAG graph from bottom to top to better understand the details of the execution plan.

<br></br>
![1.4-d1-exploding_join_shuffle_dag.png](./Includes/images/1.4-d1-exploding_join_shuffle_dag.png)

5. In the query plan, below the number of rows (199,980,000), expand the **PhotonShuffleExchangeSink** box (the arrow in the image above shows you what to expand). Notice that the **metric** for *estimated rows output* has exploded by 100x after joining the **transactions** table with the **stores** table, which is caused by duplicates of the store-id in the stores table.

6. In the same location, look at the **metric** **num bytes spilled to disk due to memory pressure total (min, med, max)**. Notice that *927.6 MiB* (value can vary) spilled to disk.

7. Leave the Spark UI open.


### D2. TODO: See the Spill
In the Spark UI view the memory spill. 

1. In the Spark UI select **Stages** on the top navigation bar. Here you can see all the stages performed on the cluster.

2. Find the stage with the largest amount of **Shuffle Writes** (should be around 1335.0 MiB, but it can vary) for the query within the **Description** column that begins with `joined_df_nobroadcast = spark("""SELECT...)`.

3. After you find that stage, select the link in the **Description** field.
![1.4-d2_find_spill.png](./Includes/images/1.4-d2_find_spill.png)

4. Look at the field **Spill (Disk)** for the stage. Notice that this query spilled to disk.

    **NOTE:** In Apache Spark, when there's more data than it can handle in memory, it puts some of the extra data on the hard drive. This is called *spilling to disk* and the 927.9 MiB means it had to move about 927.9 megabytes of data to the hard drive to keep processing.

<br></br>
![1.4-d2_memory_spill.png](./Includes/images/1.4-d2_memory_spill.png)



5. In the same page look at the **Locality Level Summary**. This means that the number of partitions in this stage is 4. Think to yourself, is this a good setting?

6. Close the Spark UI browser.


## E. Improve: Increasing the number of shuffle partitions
Let's try to increase the performance of this query by increasing the number of shuffle partitions. You can do this by modifying the **spark.sql.shuffle.partitions** configuration setting and setting it to **8** partitions. 

This configures the number of partitions to use when shuffling data for joins or aggregations.

For more information, view the [spark.sql.shuffle.partitions](https://spark.apache.org/docs/3.5.3/sql-performance-tuning.html#other-configuration-options) documentation.


In [0]:
## Rerun the cell to turn off the broadcast join features if not already turned off
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
spark.conf.set("spark.databricks.adaptive.autoBroadcastJoinThreshold", -1)

## Deal with the spill by increasing the number of shuffle partitions to 8
## Not a huge difference in this small example, but as the amount of data spilled increases
## This can make a big difference
spark.conf.set("spark.sql.shuffle.partitions", <FILL_IN>)

Run the same query as the previous example, but this time with the number of shuffle partitions set to 8. Note the time the query takes to execute.

**NOTE:** This should take about ~50 seconds.


In [0]:
joined_df_8_partitions = spark.sql("""
    SELECT 
        transactions.id,
        amount,
        countries.name as country_name,
        employees,
        stores.name as store_name
    FROM
        transactions
    JOIN
        stores
        ON
            transactions.store_id = stores.id
    JOIN
        countries
        ON
            transactions.country_id = countries.id
""")

(joined_df_8_partitions
 .write
 .mode('overwrite')
 .saveAsTable('transact_countries')
)

## Reset the shuffle.partions to the default setting
spark.conf.unset("spark.sql.shuffle.partitions")

### E1. TODO: See the Spill

In the Spark UI view the memory spill. 

1. Expand **Spark Jobs**, on the first job right click and select **Open in a New Tab**.

2. In the Spark UI select **Stages** on the top navigation bar.

3. Find the stage with the largest amount of shuffle writes (should be around 679.9 MiB, but it can vary) for the query within the **Description** column that begins with `joined_df_8_partitions = spark("""SELECT...)`.

4. After you find that stage, select the link in the **Description** field.

5. Look at the field **Spill (Disk)**. Notice that this stage spilled to disk.

    **NOTE:** In Apache Spark, when there's more data than it can handle in memory, it puts some of the extra data on the hard drive. This is called *spilling to disk* and the 273.4 MiB means it had to move about 273.4 MiB of data to the hard drive to keep processing.

    When modifying the number of partitions the spill has decreased.


<br></br>
![1.4-e_memory_spill.png](./Includes/images/1.4-e_memory_spill.png)
<br></br>

6. In the same page look at the **Locality Level Summary**. This means that the number of partitions in this stage is 6 even though we set the number of partitions to 8. This is because Spark automatically decides to combine smaller partitions into bigger ones during a job, which can help Spark finish the job more quickly and use less memory. 

    You can disable this by setting the following option `spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False)`.


7. Note down amount of data spilled has been reduced and time taken to execute the query is a bit faster. 

**NOTE:** The spill metrics of 273MB spilled can be observed in query plan in **PhotonShuffleExchangeSink** that follows the join operations, as well as stage details for stage that has largest amount of shuffle writes.

## F. Improve: Change the Order of the Join

Run the cell below to turn off **autoBroadcastJoinThreshold** if not already turned off.

In [0]:
## Turn off the broadcast join features if not already turned off
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
spark.conf.set("spark.databricks.adaptive.autoBroadcastJoinThreshold", -1)

Let's look at the modified query below, which now joins the **transactions** table with the **countries** first, then joins with the **stores** table. 

Running the smaller join first (avoiding the large exploding join) means we don't have to shuffle as much data as we did when joining with the **stores** table first.

Run the cell and note the amount of time it takes to complete the query.

**NOTE:** The query has been modified for you. Simply run the cell.

In [0]:
small_joined_first_df = spark.sql("""
    SELECT 
        transactions.id,
        amount,
        countries.name as country_name,
        employees,
        stores.name as store_name
    FROM
        transactions
    -- Notice that we are joining with countries first to avoid instead of stores --
    JOIN
        countries
        ON
            transactions.country_id = countries.id
    -- Then we join the results with the stores table, avoiding the shuffle of the large exploding join  --
    JOIN
        stores
        ON
            transactions.store_id = stores.id
""")


(small_joined_first_df
 .write
 .mode('overwrite')
 .saveAsTable('transact_countries')
)

### F1. TODO: See the Shuffles
Open the Spark UI and navigate to the query DAG. Identify the explosion of rows in the DAG of the Spark UI. To view how the DAG works, complete the following:

1. In the cell above, expand **Spark Jobs**.

2. In the first Job, right-click on **View** and select *Open in a New Tab*. 

    **NOTE:** In the Vocareum lab environment, if you click **View** without opening it in a new tab, the pop-up window will display an error.

3. In the new window, find the **Associated SQL Query** header at the top and select the number.

4. Here, you should see the entire query plan. Read the DAG graph bottom-up to better understand the details of the execution plan. 

5. In the query plan view, notice that the first join is between the **transactions** and **countries** tables, which returns 2 million results. Then, the results join with the **stores** table, creating the exploding join. This avoids having to shuffle the large join between **transactions** and **stores** (~200,000,000 rows) as we did in the first query.

<br></br>
![1.4-f_smaller_join_first_dag.png](./Includes/images/1.4-f_smaller_join_first_dag.png)

<br></br>
6. Leave the Spark UI open.


### F2. TODO: See the Spill
In the Spark UI view the memory spill. 

1. In the Spark UI select **Stages** on the top navigation bar. Here you can see all the stages performed on the cluster.

2. Find the stage with the largest amount of **Shuffle Writes** (should be around 19.1 MiB, but it can vary) for the query within the **Description** column that begins with `small_joined_first_df = spark("""SELECT...)`.

3. After you find that stage, select the link in the **Description** field.

4. Look at the field **Spill (Disk)**. Notice that this query did not spill to disk.

<br></br>
![1.4-f2_memory_spill.png](./Includes/images/1.4-f2_memory_spill.png)
<br></br>


5. Close the Spark UI browser.

Can you still see Spill in Spark UI? Did this query run faster or slower than the previous example?


## G. Improve: Analyze and Use Default Broadcast Configurations: 


Reset the **spark.sql.autoBroadcastJoinThreshold** and **spark.databricks.adaptive.autoBroadcastJoinThreshold** configurations using the `spark.conf.unset()` method.

[spark.sql.autoBroadcastJoinThreshold](https://spark.apache.org/docs/3.5.3/sql-performance-tuning.html#other-configuration-options) documentation

[spark.databricks.adaptive.autoBroadcastJoinThreshold](https://spark.apache.org/docs/3.5.3/sql-performance-tuning.html#converting-sort-merge-join-to-broadcast-join) documentation

Run the cell and view the results. Confirm the following default values for the configurations:
- *Default value of autoBroadcastJoinThreshold: 10,485,760 bytes*
- *Default value of adaptive.autoBroadcastJoinThreshold: 31,457,280 bytes*


In [0]:
# Reset defaults here
spark.conf.unset(<FILL_IN>)
spark.conf.unset(<FILL_IN>)

# Display the values
print(f'Default value of autoBroadcastJoinThreshold: {spark.conf.get("spark.sql.autoBroadcastJoinThreshold")}')
print(f'Default value of adaptive.autoBroadcastJoinThreshold: {spark.conf.get("spark.databricks.adaptive.autoBroadcastJoinThreshold")}')

Cost-based optimizers rely on statistics information to generate the most efficient physical query plan with the lowest cost. This includes decisions on join strategy and the order of joins.

Running `ANALYZE` on the joining columns across the three tables allows the optimizer to make better decisions, and everything will work as if by magic. 

Complete the cell below by writing the required `ANALYZE` statements. 

**HINT:** View the [ANALYZE TABLE](https://docs.databricks.com/en/sql/language-manual/sql-ref-syntax-aux-analyze-table.html) documentation for more information.


In [0]:

# Analyze the transactions table and compute statistics for columns country_id and store_id


# Analyze the stores table and compute statistics for the column id


# Analyze the countries table and compute statistics for the column id


In the example below, a developer writes joins without considering the optimal order of joins.

Run the query again using the initial query we used in this demonstration that joins **transactions** with **stores** first and explodes the data for the large shuffle. 

Will letting Spark figure out how to join the data efficiently work? 

Note down time taken to do the join.

In [0]:
joined_df_analyze = spark.sql("""
    SELECT 
        transactions.id,
        amount,
        countries.name as country_name,
        employees,
        stores.name as store_name
    FROM
        transactions
    JOIN
        stores
        ON
            transactions.store_id = stores.id
    JOIN
        countries
        ON
            transactions.country_id = countries.id
""")

(joined_df_analyze
 .write
 .mode('overwrite')
 .saveAsTable('transact_countries')
)

**NOTES:**
- We put the shuffle settings back to the defaults.  Usually best to stick with the defaults which tend to improve over time.  If you hard-code configurations, you may be opting out of future performance improvements unwittingly.  Always worth checking old configurations to make sure they're still needed.  You could get big performance improvements by cleaning out old configurations.

View the Spark UI. Think about the following:
- What can you see in DAG of query plan? 
- Any spill? 
- Rows exploded? 
- Order of Join? 
- Is the DAG similar to the previous joins? 
- Did the query execute faster? What were the size of the shuffle writes? 
- Larger or smaller than the previous queries?

<br></br>
**Stages**
- Notice the small amount of shuffle writes.
![1.4-g_analyze_stages.png](./Includes/images/1.4-g_analyze_stages.png)

<br></br>
**DAG**
- Look at the differences in the DAG.
![1.4_g_dag.png](./Includes/images/1.4_g_dag.png)

## Summary of the 4 Joins

| Join Strategy | Execution Time | Memory Spill | Largest Shuffle Write | Notes |
|---------------|----------------|----------------|----------------|----------------|
|D. Exploding Join| ~60 seconds| ~928 MiB |	1334.8 MiB |Joining the **transactions** table with the **store** table (exploding join) first |
|E. Increase the number of shuffles| ~50 seconds | ~273.7 MiB | 680.5 MiB |Same join as before, specify 8 partitions |
|F. Change the join order | ~40 seconds| 0 | 19.1 MiB |Change the join order to join **transactions** with **countries** first |
|G. Analyze and broadcast join| ~20 seconds| 0 | 383.6 KiB |Let Databricks analyze and use default broadcast configurations. While the query took about the same time, |



&copy; 2025 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>